In [11]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
#from  pandasgui import show
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import cross_val_predict
from sklearn.discriminant_analysis import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV

In [12]:
pd.set_option("display.max_columns", None)
warnings.filterwarnings("ignore")
train_data = pd.read_csv("../airline_passenger_satisfaction.csv")
# primeros_100_registros = train_data.iloc[:100]
# test_data = pd.DataFrame(primeros_100_registros)
#primer_registro = train_data.iloc[0]
#test_data = pd.DataFrame([primer_registro])
train_data.drop(["Unnamed: 0" ,"id"] , axis = 1 ,inplace = True)
print(train_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 23 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Gender                             103904 non-null  object 
 1   Customer Type                      103904 non-null  object 
 2   Age                                103904 non-null  int64  
 3   Type of Travel                     103904 non-null  object 
 4   Class                              103904 non-null  object 
 5   Flight Distance                    103904 non-null  int64  
 6   Inflight wifi service              103904 non-null  int64  
 7   Departure/Arrival time convenient  103904 non-null  int64  
 8   Ease of Online booking             103904 non-null  int64  
 9   Gate location                      103904 non-null  int64  
 10  Food and drink                     103904 non-null  int64  
 11  Online boarding                    1039

In [13]:
#obtenemos un arreglo con los nombres de las variables segun su tipo
imputer_cols = [cname for cname in train_data.columns if train_data[cname].dtype in ['int64', 'float64']]
categorical_cols = [cname for cname in train_data.columns if train_data[cname].dtype == "object"]

In [14]:
imputer = SimpleImputer(strategy='mean' )
imputer.fit(train_data[imputer_cols])
train_data[imputer_cols] = imputer.transform(train_data[imputer_cols])
#test_data[imputer_cols] = imputer.transform(test_data[imputer_cols])

In [15]:
# completamos valores nulos  en las columnas categoricas con la moda
def fill_null_with_mode(column, train_df):#test_df
    moda = train_df[column].mode().iloc[0]
    train_df[column] = train_df[column].fillna(moda)
    #test_df[column] = test_df[column].fillna(moda)

# Aplicar la función de llenado de valores nulos
fill_null_with_mode(categorical_cols, train_data)#test_data

In [16]:
#preparamos los datos para dividirlos
#train_data.drop(["Unnamed: 0" ,"id"] , axis = 1 ,inplace = True)
#test_data.drop(["Unnamed: 0" ,"id","satisfaction"] , axis = 1 ,inplace = True)
# TODO no estoy seguro si debo elimiar satisfaction de test
train_data


,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,Male,Loyal Customer,13.0,Personal Travel,Eco Plus,460.0,3.0,4.0,3.0,1.0,5.0,3.0,5.0,5.0,4.0,3.0,4.0,4.0,5.0,5.0,25.0,18.0,neutral or dissatisfied
1,Male,disloyal Customer,25.0,Business travel,Business,235.0,3.0,2.0,3.0,3.0,1.0,3.0,1.0,1.0,1.0,5.0,3.0,1.0,4.0,1.0,1.0,6.0,neutral or dissatisfied
2,Female,Loyal Customer,26.0,Business travel,Business,1142.0,2.0,2.0,2.0,2.0,5.0,5.0,5.0,5.0,4.0,3.0,4.0,4.0,4.0,5.0,0.0,0.0,satisfied
3,Female,Loyal Customer,25.0,Business travel,Business,562.0,2.0,5.0,5.0,5.0,2.0,2.0,2.0,2.0,2.0,5.0,3.0,1.0,4.0,2.0,11.0,9.0,neutral or dissatisfied
4,Male,Loyal Customer,61.0,Business travel,Business,214.0,3.0,3.0,3.0,3.0,4.0,5.0,5.0,3.0,3.0,4.0,4.0,3.0,3.0,3.0,0.0,0.0,satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103899,Female,disloyal Customer,23.0,Business travel,Eco,192.0,2.0,1.0,2.0,3.0,2.0,2.0,2.0,2.0,3.0,1.0,4.0,2.0,3.0,2.0,3.0,0.0,neutral or dissatisfied
103900,Male,Loyal Customer,49.0,Business travel,Business,2347.0,4.0,4.0,4.0,4.0,2.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,0.0,0.0,satisfied
103901,Male,disloyal Customer,30.0,Business travel,Business,1995.0,1.0,1.0,1.0,3.0,4.0,1.0,5.0,4.0,3.0,2.0,4.0,5.0,5.0,4.0,7.0,14.0,neutral or dissatisfied
103902,Female,disloyal Customer,22.0,Business travel,Eco,1000.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,4.0,5.0,1.0,5.0,4.0,1.0,0.0,0.0,neutral or dissatisfied


In [17]:
# Crear una copia de los datos originales para evitar modificarlos
X = train_data.drop("satisfaction", axis=1)
y = train_data["satisfaction"]

In [18]:
y

0         neutral or dissatisfied
1         neutral or dissatisfied
2                       satisfied
3         neutral or dissatisfied
4                       satisfied
                   ...           
103899    neutral or dissatisfied
103900                  satisfied
103901    neutral or dissatisfied
103902    neutral or dissatisfied
103903    neutral or dissatisfied
Name: satisfaction, Length: 103904, dtype: object

In [19]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline

# Lista de columnas numéricas
numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]

# Lista de columnas categóricas en X
categorical_cols = [cname for cname in X.columns if X[cname].dtype == "object"]

# Escalar las características numéricas
numerical_transformer = Pipeline(steps=[('scaler', MinMaxScaler())])

# Codificar one-hot las características categóricas
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(drop='if_binary', handle_unknown='ignore', sparse=False))])

# Combinar transformaciones para X
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)])


# Mapear 'satisfied' a 1 y 'neutral' a 0
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Aplicar las transformaciones a X
X = preprocessor.fit_transform(X)

# Imprimir nueva forma de X
print('Training set shape (X):', X.shape)

# Imprimir nueva forma de y
print('Training set shape (y):', y.shape)


Training set shape (X): (103904, 24)
Training set shape (y): (103904,)


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.8,test_size=0.2,random_state=42)

In [21]:
param_grid = {
    'n_estimators': range(20, 101, 20),
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'criterion' : ['gini', 'entropy']
}

list(param_grid['n_estimators'])

scoring = 'accuracy'

clf = RandomForestClassifier()
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=scoring, cv=2, verbose=2)

In [22]:
# Fit the grid search object to the training data
grid_search.fit(X_train, y_train)
 
# Print the best hyperparameters and the best score
print("Best hyperparameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)
 
# Re-train the model with the best hyperparameters
best_clf = grid_search.best_estimator_
best_clf.fit(X_train, y_train)
 
# Test the model with the best hyperparameters on the testing data
accuracy = best_clf.score(X_test, y_test)
print("Testing accuracy:", accuracy)

Fitting 2 folds for each of 90 candidates, totalling 180 fits
[CV] END criterion=gini, max_depth=None, min_samples_split=2, n_estimators=20; total time=   0.9s
[CV] END criterion=gini, max_depth=None, min_samples_split=2, n_estimators=20; total time=   0.8s
[CV] END criterion=gini, max_depth=None, min_samples_split=2, n_estimators=40; total time=   1.7s
[CV] END criterion=gini, max_depth=None, min_samples_split=2, n_estimators=40; total time=   1.5s
[CV] END criterion=gini, max_depth=None, min_samples_split=2, n_estimators=60; total time=   2.6s
[CV] END criterion=gini, max_depth=None, min_samples_split=2, n_estimators=60; total time=   2.6s
[CV] END criterion=gini, max_depth=None, min_samples_split=2, n_estimators=80; total time=   3.3s
[CV] END criterion=gini, max_depth=None, min_samples_split=2, n_estimators=80; total time=   3.3s
[CV] END criterion=gini, max_depth=None, min_samples_split=2, n_estimators=100; total time=   4.3s
[CV] END criterion=gini, max_depth=None, min_samples_sp

In [28]:
# Crear el pipeline con los parametros del grid search
my_pipeline = Pipeline(steps=[
    ('model', RandomForestClassifier(criterion = 'gini', max_depth = None, min_samples_split = 2, n_estimators = 80))
])

# Realizar la validación cruzada y obtener las probabilidades y los scores
predicted = cross_val_predict(my_pipeline, X_train, y_train, cv=5)

accuracy_scores = accuracy_score(y_train, predicted)
confusion = confusion_matrix(y_train, predicted)
recall = recall_score(y_train, predicted)
f1 = f1_score(y_train, predicted)

# Imprime las métricas para cada pliegue
print("Accuracy:", accuracy_scores)
print("Confusion Matrix:\n", confusion)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.960756950543171
Confusion Matrix:
 [[46144  1022]
 [ 2240 33717]]
Recall: 0.9377033679116723
F1 Score: 0.9538587756025801


In [24]:
classifier = RandomForestClassifier(criterion = 'gini', max_depth= None, min_samples_split=2, n_estimators=100)
classifier.fit(X_train,y_train)

RandomForestClassifier()

In [25]:
#classifier.predict(test_data)


In [26]:
y_pred = classifier.predict(X_test)

In [27]:
# Calcular promedio de las probabilidades de la clase positiva
#preds = proba_predictions[:, 1].mean()

# Calcular promedio del score de precisión
average_accuracy = accuracy_score(y, y_pred)

#Matriz
conf_matrix = confusion_matrix(y_test, y_pred)

# Calcula la precisión
precision = precision_score(y_test, y_pred, average='macro')

# Calcula el recall
recall = recall_score(y_test, y_pred, average='macro')

# Calcula el F1-Score
f1 = f1_score(y_test, y_pred, average='macro')

# Imprimir los resultados
#print("Average probability:", preds)
print("Average accuracy:", average_accuracy)
print("Matriz de Confusión:\n", conf_matrix)
print(f"Precisión: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")


ValueError: Found input variables with inconsistent numbers of samples: [103904, 20781]

**Average probability**  
Esto nos indica que de la cantidad de clientes satisfechos (satisfied) hay un 43.35% del total general , por lo tanto lo restante serian los que estan como "neutral or dissatisfied".

**Average accuracy**  
Aqui podemos observar que en cuanto a predicciones correctas, es decir que nuestro modelo cuenta con un 96.15% de acertividad en cuanto a las pruebas realizadas.

**Matriz de confusión**  
La matriz de confusión es la variable que nos permite determinar la capacidad del modelo para evitar clasificar incorrectamente las instancias negativas como positivas. A diferencia de la precisión uqe nos da una vision general de la acertividad , la matriz de confusión se utiliza para saber la categorizacion de las predicciones.

En esta métrica nos indica cuantos verdaderos positivos hay (11510) es decir que la mayoria de resultados aqui estan estado "satisfied"; Falsos positivos aqui se puede ver que el modelo interpretó 266 clientes que en realidad estan en estado  "neutral or dissatisfied" pero que el modelo tomó como "satisfied"; En cuanto a 579 , nos indica que el modelo interpreta estos datos como falsos negativos , que quiere decir que en realidad estos clientes estan como "satisfied" pero el modelo los tomó como "neutral or dissatisfied"; por último 8426 que significa que aqui estan los verdaderos negativos , que aqui todos los clientes estan como "neutral or dissatisfied".

**Precisión**  
Esto nos indica que de toda la base de datos el 96% es acertado con respecto a predicciones positivas, a diferencia de ka atriz de confusión , la precisión no tiene en cuenta estos falsos positivos y se centra en la proporción de predicciones positivas correctas en relación con todas las predicciones positivas. 

**Recall**  
El recall responde a la pregunta: "De todos los casos positivos reales, ¿cuántos de ellos el modelo fue capaz de identificar correctamente?". Es una métrica importante en problemas donde la detección de todos los casos positivos es crítica, como en la detección temprana de enfermedades o la identificación de fraudes. En este caso se puede determinar que el 96% de de los casos verdaderos positivos es correctamente identificado por el modelo.

**F1-Score**  
El F1-Score es útil cuando deseamos tener un equilibrio entre la precisión y el recall. Proporciona una puntuación que combina ambas métricas y es útil cuando ninguna de las dos métricas por sí sola es suficiente. En nuestro caso indica que está cerca al 1.0 entonces indica un buen equilibrio entre la precisión y el recall.


In [ ]:
# Obtiene la importancia de las características
importancia_caracteristicas = classifier.feature_importances_

# Puedes imprimir la importancia de cada característica
for i, importancia in enumerate(importancia_caracteristicas):
    print(f'Característica {i}: {importancia}')

In [ ]:
# Mapea las etiquetas a valores binarios
y_true_binary = [1 if label == "satisfied" else 0 for label in y_test]

# Obtén las probabilidades de predicción del modelo (por ejemplo, un clasificador de bosque aleatorio)
probs = classifier.predict_proba(X_test)  # X_test son las características de prueba

# Calcula la curva ROC especificando pos_label=1
fpr, tpr, umbrales = roc_curve(y_true_binary, probs[:, 1], pos_label=1)

# Calcula el área bajo la curva ROC (AUC-ROC)
roc_auc = auc(fpr, tpr)

# Grafica la curva ROC
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()

#### Analisis de curva ROC 

En la curva de ROC podremos notar como el modelo tiene un 0.99 de acertividad a la hora de clasificar, sabiendo distinguir correctamente entre valores correctos e incorrectos con respecto al humbral. La curva de satisfechos y la curva de no satisfechos no se superponen casi en lo absoluto, permitiendo al modelo una facil identificacion y teniendo un minimo error al distinguir entre uno o el otro. 